# jupyter wire-meta-kernel in nodejs

Yes, please.



## introduction

Done: implement both meta kernels in python and socket based kernels 

Done: rewrite entire nodejs kernel functionality in one notebook and a few lines of code

TODO: submit to this page and exported notebook of the kernel?



https://github.com/jupyter/jupyter/wiki/Jupyter-kernels

There are 2 main ways to implement a jupyter kernel, by overriding the do_execute, do_inspect, do_complete, etc methods derived from IPython.kernel

https://jupyter-client.readthedocs.io/en/stable/wrapperkernels.html

OR:

https://jupyter-client.readthedocs.io/en/stable/messaging.html#wire-protocol

The way [nel](https://github.com/n-riesco/nel), [jupyter-nodejs](https://github.com/notablemind/jupyter-nodejs), [jp-babel](https://github.com/n-riesco/jp-babel), [jp-kernel](https://github.com/n-riesco/jp-kernel), [ijavascript](https://github.com/n-riesco/ijavascript), and many others in many languages work, by implementing the ZMQ wire protocol, and pretty much calling the same execution flow.


I will attempt to explain the control flow as best I can:


## notebook kernel json?

The first step of the kernel is to install the definition so it can be recognized by Jupyter. This is a command to run the kernel in it's NATIVE language.


### get kernel json?


In [ ]:
var path = require('path');
var importer = require('../Core');
var interface = importer.import('enforcing an interface');

var jsonInterface = {
    display_name: '',
    argv: [], // not optional
    language: '', // not optional
    metadata: [void 0, {}], // optional
    env: [void 0, {}], // optional TODO dictionary descriptor types?
    interrupt_mode: [void 0, ''] // optional
}

function wireJson(kernel_json) {
    var wire_json = Object.create(interface(kernel_json, jsonInterface))
    Object.assign(wire_json, {
        argv: (kernel_json.argv || [])
            .filter(a => a.includes('{connection_file}')).length === 0
            ? (kernel_json.argv || []).concat(['{connection_file}'])
            : (kernel_json.argv || [])
    })
    var self = Object.create(wire_json)
    return self;
}

// TODO: use yargs to parse from string
function pathJson(kernel_json) {
    return interface(Object.assign({
        argv: (kernel_json.path
               ? [kernel_json.path]
               : []).concat(kernel_json.argv || kernel_json.args || [])
    }, kernel_json), jsonInterface)
}

module.exports = {
    jsonInterface,
    wireJson,
    pathJson,
};


#### test notebook kernel json?


In [ ]:
var importer = require('../Core');
var interface = importer.import('enforcing an interface');
var {wireJson, jsonInterface} = importer.import('notebook kernel json');

var json = wireJson({
    display_name: 'Node JS',
    argv: [],
    language: '',
    some_other_stuff: true
});
console.log(JSON.stringify({
    argv: json.argv,
    display_name: json.display_name,
    language: json.language,
}))
console.log(interface(json, jsonInterface));

/* output like 
{ argv:
   [ 'npm',
     'run',
     'import',
     '--',
     'get javascript kernel',
     '["{connection_file}"]' ] }
*/


### custom kernel json?




In [ ]:
var importer = require('../Core');
var {jsonInterface, wireJson, pathJson} = importer.import('notebook kernel json');

function pythonJson(kernel_json) {
    return pathJson(Object.assign({
        argv: ['python3', '-m', 'IPython.kernel', '-f', '{connection_file}']
    }, kernel_json))
}

// derrived from https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js

function nodeJson(kernel_json) {
    return pathJson(Object.assign({
        path: process.argv[2] || process.argv[0],
        argv: [kernel_json.path].concat(kernel_json.argv || kernel_json.args || [])
    }, kernel_json))
}

function notebookJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: ['npm', 'run', 'import', '--', 'get javascript kernel',
               '["{connection_file}"]'],
    }, kernel_json))
}

function bashJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: ['npm', 'run', 'import', '--', 'get bash kernel',
               '["{connection_file}"]'],
    }, kernel_json))
}

function processingJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: ['npm', 'run', 'import', '--', 'get processing kernel',
               '["{connection_file}"]'],
    }, kernel_json))
}

module.exports = {
    pythonJson,
    nodeJson,
    notebookJson,
    bashJson,
    processingJson
};


## jupyter meta kernel?

This meta-kernel runs inside a node process that is started and managed by the wire protocol kernel.

Node specific functionality has been extracted so that it is easy to translate between languages.

Wire meta kernel splits up the meta kernel control flow from the wire request kernel control flow.


### kernel language interface?

language kernel information 

https://jupyter-client.readthedocs.io/en/stable/messaging.html#kernel-info


In [ ]:
var languageInterface = {
    mimetype: '',
    name: '',
    file_extension: '',
    version: [void 0, null, ''],
    pygments_lexer: [void 0, null, ''],
    codemirror_mode: [void 0, null, ''],
    nbconvert_exporter: [void 0, null, '']
}

module.exports = languageInterface;


### kernel info interface?
 
Kernel info is just the static properties required by the meta kernel implementation. These are also returned by the kernel_info_request in the wire protocol implementation

https://jupyter-client.readthedocs.io/en/stable/messaging.html#kernel-info


In [ ]:
var importer = require('../Core');
var interface = importer.import('enforcing an interface');
var languageInterface = importer.import('kernel language interface');

var PACKAGE_VERSION = require('../package.json').version;

var kernelInfoInterface = {
    // from the docs 
    protocol_version: [void 0, ''],
    implementation: [void 0, ''],
    implementation_version: [void 0, ''],
    banner: [void 0, ''],
    language_info: languageInterface,
    help_links: [void 0, []],
    // custom, path to it's own script for installing
    install_config: [void 0, '']
}

function nativeKernelInfo(kernel_info) {
    if(!kernel_info) {
        return;
    }
    var {
        language_info,
        protocol_version,
        implementation,
        implementation_version,
        banner
    } = kernel_info; // for easy reading
    
    //if(typeof language_info === 'function')
    //    kernel_info.language_info = language_info(language_info);
    kernel_info.language_info = interface(language_info, languageInterface);
    language_info.name = language_info.name || implementation;
    var info = interface(kernel_info, kernelInfoInterface)
    var self = extend(meta_info, {
        protocol_version: protocol_version || '5.1',
        implementation: implementation || language_info.name,
        implementation_version: implementation_version || PACKAGE_VERSION,
        banner: banner || language_info.language
    })
    return self;
}

module.exports = {
    kernelInfoInterface,
    nativeKernelInfo
};


### meta kernel interface?



In [ ]:
var importer = require('../Core');
var languageInterface = importer.import('kernel language interface');
var {kernelInfoInterface} = importer.import('kernel info interface');

var metaKernelInterface = {
    ...kernelInfoInterface,
    language_info: languageInterface,
    kernel_info: kernelInfoInterface,
    kernel_config: [void 0, {}],
    do_init: () => {}, // custom for executing once
    do_message: () => {}, // custom input message
    do_respond: () => {}, // custom output message
    do_execute: () => {},
    do_complete: [void 0, () => {}],
    do_inspect: [void 0, () => {}],
    do_history: [void 0, () => {}],
    do_is_complete: [void 0, () => {}],
    do_shutdown: [void 0, () => {}],
    // custom for executing install script, even remotely!
    do_install: () => {},
}

module.exports = metaKernelInterface;


### native meta kernel?



In [ ]:
var importer = require('../Core');
var {extend} = importer.import('extend prototype class');
var interface = importer.import('enforcing an interface');
var metaKernelInterface = importer.import('meta kernel interface');
var nativeMethods = importer.import('native meta kernel methods');
var {nativeKernelInfo} = importer.import('kernel info interface');

// automatically fill in some properties passed in from children
function reassignProperties(meta_kernel) {
    var {
        language_info,
        kernel_config,
        kernel_info,
        install_config
    } = meta_kernel;
    
    if(!language_info && kernel_info)
        meta_kernel.language_info = kernel_info.language_info
    
    // if(typeof kernel_config === 'function')
    //     meta_kernel.kernel_config = kernel_config(kernel_config);
    if(kernel_info && !kernel_info.language_info)
        kernel_info.language_info = language_info;
    
    meta_kernel.kernel_info = nativeKernelInfo(kernel_info);
    
    // Only auto init if receiving a valid connection file?
    if(fs.existsSync(kernel_config)) {
        meta_kernel.kernel_config = Object.assign(
            {autoinit: true},
            meta_kernel.start_config || {},
            JSON.parse(fs.readFileSync(kernel_config)),
        );
    }
    
    if(typeof install_config === 'function')
        meta_kernel.install_config = install_config({
            display_name: install_config.display_name
                || meta_kernel.banner,
            language: install_config.language
                || meta_kernel.implementation,
        });
}

function nativeMetaKernel(meta_kernel) {
    reassignProperties(meta_kernel);
    var meta = interface(meta_kernel, metaKernelInterface);
    var kernel = extend(meta, nativeMethods);
    
    // TODO: allow calling any kernel from command line with a repl interface
    // reference jupyter command for this
    if(this.kernel_config === 'do_install') {
        kernel.do_install(kernel.install_config);
    } else if((kernel.kernel_config || {}).autoinit) {
        kernel.do_init(kernel.kernel_config);
    } 
    return kernel;
}

module.exports = nativeMetaKernel;


### native meta methods?



In [ ]:
var importer = require('../Core');
var mkdirpSync = importer.import('mkdirp');

function do_install(configJson) {
    console.log(`installing kernel ${JSON.stringify(configJson)}`);
    mkdirpSync(`./.kernel/${configJson.language}`);
    require('fs').writeFileSync(`./.kernel/${configJson.language}/kernel.json`,
                                JSON.stringify(configJson, null, 4));
// TODO: convert to bash notebook or kernel call
    require('child_process').execSync(`
jupyter kernelspec install --user --replace "./.kernel/${configJson.language}"`);
    require('rimraf').sync('./.kernel');
}

function do_message(message) {
    if(typeof this[Object.keys(message)[0]] === 'undefined') {
        console.error(`unhandled message type ${JSON.stringify(message)}`);
        return;
    }
    var result = this[Object.keys(message)[0]].apply(this, Object.values(message));
    if(result && Object.getPrototypeOf.constructor === Promise) {
        
    }
    return result;
}

function do_respond(message) {
    // TODO: add Promise handling here?
    // TODO: add line and block magics handling here?
    return message;
}

function do_is_complete(message) {
    try {
        this.do_execute(message);
        return true;
    } catch (e) {
        return false;
    }
}
    
function do_init(config) {
    console.log('starting meta kernel');
     // incase different than kernel_info.kernel_config
    this.kernel_config = config;
    
}

module.exports = {
    do_init,
    do_is_complete,
    do_message,
    do_respond,
    do_install,
}


## process meta kernel

Add stdio processing to the kernel.


### socket meta kernel?

simple meta kernel for deriving stdio information.


In [ ]:
var importer = require('../Core');
var interface = importer.import('enforcing an interface');
var metaKernelInterface = importer.import('meta kernel interface');
var nativeMetaKernel = importer.import('native meta kernel');
var nativeMethods = importer.import('native meta kernel methods');
var {extend} = importer.import('extend prototype class');
var assert = require('assert');

function do_init(config) {
    nativeMethods.do_init.call(this, config);
    console.log('starting socket kernel');
    if(this.socket) {
        this.socket.on('message', this.do_message.bind(this));
// TODO: this is a process feature but should be in the socket class?
/*Object.assign(global.console, {
    log: this.socket.stdout.write,
    error: this.socket.stderr.write,
    warn: this.socket.stderr.write,
});*/
    } else {
        throw new Error('socket not ready!');
    }
}

// TODO: do something on the client when shutting down
//   same thing as parent but with process?
// do_shutdown: (kernel, message) => kernel.socket.send({do_shutdown: message}),

function do_respond(message) {
    message = nativeMethods.do_respond.call(this, message);
    if(this.socket.send) {
        message = {do_respond: {content: message}};
        return this.socket.send(message);
    }
    // assert here, possible defect, not sure when this would be used?
    assert(false, 'why is this used?');
    return message;
}

function socketMetaKernel(meta_kernel) {
    // every time meta_kernel is extended we interface the input
    var meta = interface(meta_kernel, metaKernelInterface);
    //  to get the assignment values for the child object
    var kernel = extend(meta, {
        do_init,
        do_respond
    })
    return nativeMetaKernel(kernel);
}

module.exports = {
    socketMetaKernel,
    do_respond,
    do_init
};



### process meta kernel?

simple kernel that extends the socket and adds a child process.


In [ ]:
var {spawn} = require("child_process");
var importer = require('../Core');
var interface = importer.import('enforcing an interface');
var metaKernelInterface = importer.import('meta kernel interface');
var {extend} = importer.import('extend prototype class');
var {socketMetaKernel} = socketMethods = importer.import('socket meta kernel')
var nativeMethods = importer.import('native meta kernel methods');

function processMetaKernel(meta_kernel) {
    var meta = interface(meta_kernel, metaKernelInterface);
    var kernel = extend(meta, {
        do_init,
        do_shutdown,
        do_message,
        do_execute,
        do_complete,
        do_inspect,
        do_history,
        do_is_complete,
    })
    return socketMetaKernel(kernel);
}

function do_execute(message) { return this.do_message(message, do_execute)}
function do_complete(message) { return this.do_message(message, do_complete)}
function do_inspect(message) { return this.do_message(message, do_inspect)}
function do_history(message) { return this.do_message(message, do_history)}
function do_is_complete(message) { return this.do_message(message, do_is_complete)}

function do_message(message, func) {
    if(!this.socket) {
        throw new Error('socket not ready!')
    }
    if(typeof func === 'undefined') {
        return nativeMethods.do_message.call(this, message);
    }
    // TODO: add execution do_respond method handling to this layer instead
    var client = {};
    client[func.name] = {content: message};
    this.socket.send(client);
}

// TODO: move this to socket kernel and call from parent process kernel
function do_shutdown(message) {
    return Promise.resolve()
        .then(() => this.socket.kill('SIGTERM'))
        .then(() => process.exit() /*request.content.restart
              ? kernel.do_init(kernel.kernel_config, kernel)
              : void 0*/)
}

function do_init(config) {
    var {child_process} = config; // for readability
    if(!child_process) {
        throw new Error(`meta_kernel not implemented! ${
                        JSON.stringify(child_process)}`);
    }
    if(typeof child_process === 'string') {
        child_process = [child_process];
    }
    console.log('spawning child process');
    this.socket = spawn(child_process[0],
                        child_process.slice(1),
                        {
                            cwd: config.cwd || '.',
                            stdio: ['pipe', 'pipe', 'pipe', 'ipc']
                        //    stdio: [0, 1, 2]
                        //    stdio: ['ignore', 'ignore', 'ignore']
                        //    stdio: [process.stdin,
                        //            process.stdout,
                        //            process.stderr, 'ipc']
                        })
    socketMethods.do_init.call(this, config)
}

module.exports = {
    do_init,
    do_shutdown,
    do_message,
    do_execute,
    do_complete,
    do_inspect,
    do_history,
    do_is_complete,
    processMetaKernel
};



#### test process meta kernel?



In [ ]:
var importer = require('../Core');
var {processMetaKernel} = importer.import('process meta kernel')
var metaKernelInterface = importer.import('meta kernel interface');
var interface = importer.import('enforcing an interface');

function testProcessKernel() {
    return new Promise(resolve => {
        var kernel = processMetaKernel({
            do_respond: (message) => {
                console.log(message);
                resolve(kernel);
            },
            kernel_config: {
                autoinit: true,
                child_process: [
                    'node', '--eval', `
    var importer = require('./Core');
    var nodeMetaKernel = importer.import('node meta kernel')
    nodeMetaKernel({
        kernel_config: {autoinit: true}
    })
    process.stdin.resume();
`
                ]
            },
        });
        setTimeout(() => {
            //console.log(interface(kernel, metaKernelInterface, false))
            console.log(kernel.do_message({do_execute: 'console.log(1 + 1)'}));
            console.log(kernel.do_message({do_execute: '1 + 1'}));
        }, 5000)
    })
    .then(kernel => console.log(kernel.do_shutdown()))
}

module.exports = testProcessKernel;

if(typeof $$ !== 'undefined') {
    $$.async();
    testProcessKernel()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
    
    /* expected output
    undefined
    undefined
    2
    2
    */

}


### TODO: repl process kernel?

Add response tracing, stdio/console streaming, and fancy display messages.


In [ ]:
var importer = require('../Core');
var {processMetaKernel} = processMethods = importer.import('process meta kernel')

function do_init(config) {
    processMethods.do_init.call(this, config);
    this.socket.stdout.on('data', this.do_respond);
    this.socket.stderr.on('data', this.do_respond);
}

function do_respond(message) {

/* TODO: streams
{stdout: {
    name: stream,
    text: text,
}})
*/
    // TODO: 
    // TODO: call reply methods
    // no need to call socketMethods.do_respond again!
}

function replMetaKernal(meta_kernel) {
    var meta = interface(meta_kernel, metaKernelInterface);
    var kernel = extend(meta, {
        do_respond,
        do_init
    })
    return processMetaKernel(kernel);

}

module.export = {
    replMetaKernal,
    do_respond,
    do_init
}



#### test repl process using bash?



In [ ]:
var importer = require('../Core');
var {processMetaKernel} = processMethods = importer.import('process meta kernel')
var metaKernelInterface = importer.import('meta kernel interface');
var interface = importer.import('enforcing an interface');

function do_respond(message) {
    console.log(`response "${message.toString()}"`);
}

// TODO: move this to ibash kernel below
function testProcessKernel() {
    return new Promise(resolve => {
        var kernel = bashKernel({
            do_respond: (message) => {
                setTimeout(resolve.bind(kernel, kernel), 1000);
                return do_respond(message);
            },
            kernel_config: {
                autoinit: true, 
                // because its a test with no connection file
                //   so native doesn't automatically start
            }
        })

        setTimeout(() => {
            console.log(interface(kernel, metaKernelInterface, false))
            console.log(kernel.do_message({do_execute: `hello`}))
        }, 500)
    })
    .then(kernel => console.log(kernel.do_shutdown()))
}

module.exports = testProcessKernel;

if(typeof $$ !== 'undefined') {
    $$.async();
    testNodeProcessKernel();
    
    /* expected output
    undefined
    undefined
    2
    2
    */

}


## wire kernel




partly derrived from:

https://github.com/n-riesco/nel/blob/master/lib/nel.js

https://github.com/n-riesco/ijavascript/blob/master/lib/kernel.js

https://github.com/notablemind/jupyter-nodejs/blob/master/lib/kernel.js

https://github.com/n-riesco/jp-kernel/blob/master/lib/jp-kernel.js

these functions are split up to help distinguish between control flow and language specific functionality.

if the wire kernel methods are not defined, the response is sent directly from the native kernel to the front-end



### jupyter wire interface?

Implements all message types

https://jupyter-client.readthedocs.io/en/stable/messaging.html

Adds `*_reply` methods for calling from the subprocess meta_kernel and the higher level node wire kernel.




In [ ]:
var importer = require('../Core');
var metaKernelInterface = importer.import('meta kernel interface');

var wireKernelInterface = {
    ...metaKernelInterface,
    // implement all requests, these are required by the meta kernel
    execute_request: () => {},
    inspect_request: () => {},
    complete_request: () => {},
    shutdown_request: () => {},
    history_request: () => {},
    is_complete_request: () => {},
    
    // requests maybe not required right now?
    kernel_info_request: () => {},
    interrupt_request: [void 0, () => {}],
    input_request: [void 0, () => {}],
    connect_request: [void 0, () => {}],
    comm_info_request: [void 0, () => {}],

    // custom, implement all replys,
    //   allows for manipulating before sending
    execute_reply: [void 0, () => {}],
    inspect_reply: [void 0, () => {}],
    complete_reply: [void 0, () => {}],
    shutdown_reply: [void 0, () => {}],
    history_reply: [void 0, () => {}],
    is_complete_reply: [void 0, () => {}],
    
    kernel_info_reply: [void 0, () => {}],
    connect_reply: [void 0, () => {}],
    comm_info_reply: [void 0, () => {}],
    interrupt_reply: [void 0, () => {}],
    input_reply: [void 0, () => {}],

    // a few extra protocol methods
    //   these might just be used to intercept a message from the client
    display_data: [void 0, () => {}],
    update_display_data: [void 0, () => {}],
    execute_input: [void 0, () => {}],
    execute_result: [void 0, () => {}],
    error: [void 0, () => {}],
    status: [void 0, () => {}],
    clear_output: [void 0, () => {}],
    comm_msg: [void 0, () => {}],
    comm_close: [void 0, () => {}],
}

module.exports = wireKernelInterface;
    

### TODO: stdio and status messages

TODO: Add wire features one level at a time




In [ ]:
    status: (kernel, request) => kernel.do_respond({
        status: request.content
    }),

    function onError(result) {
        request.respond(
            this.shellSocket,
            "execute_reply", {
                status: "error",
                execution_count: this.executionCount,
                ename: result.error.ename,
                evalue: result.error.evalue,
                traceback: result.error.traceback,
            }
        );

        request.respond(
            this.iopubSocket,
            "error", {
                execution_count: this.executionCount,
                ename: result.error.ename,
                evalue: result.error.evalue,
                traceback: result.error.traceback,
            }
        );
    }

    function onStdout(data) {
        request.respond(
            this.iopubSocket,
            "stream", {
                name: "stdout",
                text: data.toString(),
            }
        );
    }

    function onStderr(data) {
        request.respond(
            this.iopubSocket,
            "stream", {
                name: "stderr",
                text: data.toString(),
            }
        );
    }



### TODO: wire message routing

Call the socket translation methods from the Tools section and send the specific response types over the right channel.



In [ ]:
do_init: (kernel, config) => {
    console.log('starting wire sockets');
    return setupSockets(config)
        .then(bindSockets)
        .then(() => kernel.meta_kernel.do_init(kernel, config))
        .then(() => this.do_respond(kernel, {
            status: {execution_state: 'idle'}
        }));
},
do_respond: (kernel, message) => {
    var result = {};
    var msg_type = Object.keys(message)[0];
    result[msg_type] = {content: message[msg_type]};
    return wireRespond(kernel, result);
},
do_message: wireMessage,
// TODO: bubble response messages from child to front-end

function bindSockets(sockets) {
    console.log('connecting sockets');
    this.sockets = sockets;
    sockets.heartbeat.on('message', sockets.heartbeat.send);
    [sockets.control, sockets.shell, sockets.stdin]
        .forEach(socket => socket.on('message', addCB.bind(
            null,
            parseMessage,
            parsed => this.do_message(this, parsed))))
    // iopub appears to be write to only
}

function wireRespond(kernel, message) {
    console.log(`response`, message);
    var msg_type = Object.keys(message)[0];
    var encoded = collapseMessage(kernel.kernel_config.key, message);
    if(msg_type === 'shutdown_reply')
        kernel.sockets.control.send(encoded);
    else if(msg_type.substr(-6) === '_reply')
        kernel.sockets.shell.send(encoded);
    else if(msg_type === 'input_request')
        kernel.sockets.stdin.send(encoded);
    else
        kernel.sockets.iopub.send(encoded);
}

function wireMessage(kernel, message) {
    var msg_type = Object.keys(message)[0];
    console.log(message);
    // TODO: need to pass arguments, that's why callback was used
// TODO: add execution count recorder, also to do_respond maybe
    /*
    kernel.do_respond = (message) => kernel.do_respond(
        kernel, message,
        
    
    // assign the responder
    var result = {};
    var execution_count = Object.values(message)[0].execution_count;
    delete Object.values(message)[0].execution_count;
    result[Object.keys(message)[0]] = {
        execution_count,
        content: Object.values(message)[0],
        respond: responders[execution_count]
    };

    */
    nativeMetaKernel.prototype.do_message(Object.assign({}, kernel, {
        do_respond: (response) => {
            var result = {};
            var res_type = Object.keys(response)[0];
            result[res_type] = {
                parent: message[msg_type].header,
                metadata: message[msg_type].metadata,
                content: response[res_type]
            };
            return wireRespond(kernel, result);
        }
    }), message);
}


### TODO jupyter wire kernel?


In [ ]:
var importer = require('../Core');
var extend = importer.import('extend prototype')
var {
    setupSockets, parseMessage, collapseMessage
} = importer.import('decode encode ipython zmq protocol');
var {nativeMetaKernel, metaKernelInterface} = importer.import('jupyter meta kernel');
var {processMetaKernel} = importer.import('process meta kernel');
var {wireMetaKernel} = importer.import('wire meta kernel');

// TODO: move this to patterns, improve this flow
function addCB(og, cb) {
    return cb(og.apply(null, Array.from(arguments).slice(2)));
}

function wireKernel(kernel) {
    console.log('starting wire kernel');
    console.log(kernel.kernel_info);
    console.log(this.kernel_info);
    
    var wire_kernel = extend(kernel, wireKernelInterface);
    console.log(wire_kernel.kernel_info);
    
    var meta_kernel = extend(wire_kernel, wireMetaKernel);
    console.log(meta_kernel.kernel_info);
    
    return new processMetaKernel(meta_kernel);
}

wireKernel.prototype = {
    input_request: (kernel, request) => {
        // TODO: finish this
        //this.onReplies[response.header.msg_id] = onReply;
    },
    comm_info_request: (kernel, request) => {
        kernel.do_respond({status: {execution_state: 'busy'}});
        kernel.do_respond({status: {execution_state: 'idle'}});
        kernel.do_respond({comm_info_reply: {comms: {}}});
    },
    kernel_info_request: (kernel, request) => {
        kernel.do_respond({status: {execution_state: 'busy'}});
        kernel.do_respond({status: {execution_state: 'idle'}});
        kernel.do_respond({kernel_info_reply: kernel.kernel_info});
    },
}

module.exports = {
    wireKernelInterface,
    wireKernel
}


### TODO: wire meta kernel?

Respond to all the meta type requests with the necessary control flow. Converts wire kernel control to meta kernel messaging.


In [ ]:
var importer = require('../Core');
var extend = importer.import('extend prototype')

var count = 0;
var responders = [];

function do_execute(kernel, request) {
    // TODO: do some magics here?
    var execution_count = ++count;
    responders[execution_count] = kernel.do_respond;
    kernel.do_respond({status: {execution_state: 'busy'}});
    kernel.do_respond({execute_input: {
        execution_count: request.execution_count,
        code: request.content.code,
    }})
    var result;
    return Promise.resolve()
        .then(() => kernel.do_execute(request.content))
        .then(r => result = r)
        .then(() => kernel.do_respond({status: {execution_state: 'idle'}}))
        .then(() => kernel.do_respond({execute_reply: {
            status: 'ok',
            execution_count: request.execution_count,
            payload: [], // TODO(NR) not implemented,
            user_expressions: {}, // TODO(NR) not implemented,
        }}))
        .then(() => kernel.do_respond({execute_result: {
            execution_count: request.execution_count,
            data: {'text/plain': result + ''},
            metadata: {}
        }}))
}

function do_display(kernel, request) {
    kernel.do_respond(request.display_id
        ? {update_display_data: {
            metadata: {},
            data: request.content,
            transient: request.display_id}}
        : {display_data: {metadata: {}, data: request.content}})
}

function do_shutdown(kernel, request) {
    kernel.do_respond({status: {execution_state: 'busy'}});
    return Promise.resolve()
        .then(() => kernel.do_shutdown(request.content))
        .then(r => result = r)
        // TODO: some sort of shutdown scripting
        .then(() => kernel.do_respond({status: {execution_state: 'idle'}}))
        .then(() => kernel.do_respond({shutdown_reply: request.content}))
}

function do_complete(kernel, request) {
    kernel.do_respond({status: {execution_state: 'busy'}});
    var result;
    return Promise.resolve()
        .then(() => kernel.do_complete(request.content))
        .then(r => result = r)
        .then(() => kernel.do_respond({status: {execution_state: 'idle'}}))
        .then(() => kernel.do_respond({complete_reply: {
            matches: result.completion.list,
            cursor_start: result.completion.cursorStart,
            cursor_end: result.completion.cursorEnd,
            status: "ok",
        }}))
}

function do_history(kernel, request) {
    kernel.do_respond({status: {execution_state: 'busy'}});
    return Promise.resolve()
        .then(() => kernel.do_history(request.content))
        .then(r => result = r)
        .then(() => kernel.do_respond({status: {execution_state: 'idle'}}))
        .then(() => kernel.do_respond({history_reply: {
            history: [] // TODO
        }}))
}

function do_is_complete(kernel, request) {
    kernel.do_respond({status: {execution_state: 'busy'}});
    var result;
    return Promise.resolve()
        .then(() => kernel.do_is_complete(request.content))
        .then(r => result = r)
        .then(() => kernel.do_respond({status: {execution_state: 'idle'}}))
        .then(() => kernel.do_respond({is_complete_reply: {
            status: (result ? 'complete': 'incomplete'),
            indent: ''
        }}))
}

function do_inspect(kernel, request) {
    var execution_count = ++count;
    kernel.do_respond({status: {execution_state: 'busy'}});
    var result;
    return Promise.resolve()
        .then(() => kernel.do_inspect(request.content))
        .then(r => result = r)
        .then(() => kernel.do_respond({status: {execution_state: 'idle'}}))
        // TODO: move this to do_display method?
        .then(() => result.inspection
              ? result.inspection.type + ': ' + result.inspection.string
              : result.doc.usage
                  ? result.doc.usage + '\n\n' + result.doc.description
                  : result)
        // TODO: use error for all promises
        .then(() => kernel.do_respond({inspect_reply: {
            found: true,
            data: {'text/plain': result, 'text/html': `<pre>${result}</pre>`},
            metadata: {},
            status: "ok",
        }}))
        .catch(e => kernel.do_respond({inspect_reply: {
            status: "error",
            execution_count: execution_count,
            ename: result.error.ename,
            evalue: result.error.evalue,
            traceback: result.error.traceback,
        }}))
}

function wireMetaKernel(meta_kernel) {
    return extend(metaKernelInterface(session), wireMetaKernel);
}

wireMetaKernel.prototype = {
    shutdown_request: do_shutdown,
    is_complete_request: do_is_complete,
    execute_request: do_execute,
    complete_request: do_complete,
    history_request: do_history,
    inspect_request: do_inspect,
};

module.exports = {
    wireMetaKernel
}


## node kernel



### node meta kernel?

derrived from:

https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js



In [ ]:
var vm = require('vm');
var importer = require('../Core');
var {extend} = importer.import('extend prototype class');
//var {processMetaKernel} = importer.import('process meta kernel')
var nativeMetaKernel = importer.import('native meta kernel');
var nativeMethods = importer.import('native meta kernel methods');
var {socketMetaKernel} = socketMethods = importer.import('socket meta kernel')

var TRANSFORM = {
    presets: [
        [require.resolve("@babel/preset-env"), {
            loose: true,
            targets: {node: true},
        }],
    ],
}

function do_execute(code) {
    // TODO: add Promise handling here?
    var transpiled = this.transpile(code, TRANSFORM).code;
    var result = vm.runInThisContext(transpiled);
    return this.do_respond(result); // TODO: move do_repond
}

// overrides try/catch based method from parent
//   because it has a syntax checker
function do_is_complete(code) {
    console.log('checking syntax')
    var transpiled = this.transpile(code, TRANSFORM).code;
    return this.do_respond(vm.Script(transpiled)); // TODO: move do_repond
}

function do_init(config) {
    this.socket = process;
    socketMethods.do_init.call(this, config);
    console.log('starting node meta kernel');
    this.transpile = require("@babel/core").transform;
}

function nodeMetaKernel(meta_kernel) {
    var node_meta = extend(meta_kernel, {
        do_execute,
        do_is_complete,
        do_init,
    })
    return socketMetaKernel(node_meta);
}

module.exports = nodeMetaKernel;


#### test node meta kernel?

In [ ]:
var importer = require('../Core');
var nodeMetaKernel = importer.import('node meta kernel')
var metaKernelInterface = importer.import('meta kernel interface');
var interface = importer.import('enforcing an interface');

function testNodeMetaKernel() {
    var kernel = nodeMetaKernel({
        kernel_config: {autoinit: true}
    });
    console.log(interface(kernel, metaKernelInterface, false))
    console.log(kernel.do_message({do_execute: 'console.log(1 + 1)'}));
    console.log(kernel.do_message({do_execute: '1 + 1'}));
}

module.exports = testNodeMetaKernel;

if(typeof $$ !== 'undefined') {
    testNodeMetaKernel();
    
    /* expected output
    2
    undefined
    2
    */

}


### node wire kernel?


In [ ]:
var importer = require('../Core');
var {nodeJson, notebookJson} = importer.import('get kernel json');
var {wireKernel} = importer.import('jupyter wire kernel');
var {nativeKernelInfo} = importer.import('get kernel info');
var {processMetaKernel} = importer.import('process meta kernel');
var {languageInterface} = importer.import('get kernel language');

function getVersion(str) {
    return str.split('.').map(v => parseInt(v, 10)).join('.')
}

function nodeKernel(config, options) {
    return wireKernel({
        kernel_config: config,
        start_config: options,
        install_config: notebookJson,
        kernel_info: {
            banner: 'Node JS',
            // TODO: automatically create this from installation intructions
            help_links: ['https://nodejs.org']
        },
        language_info: {
            name: 'node',
            file_extension: '.js',
            mimetype: 'application/javascript',
            version: getVersion(process.versions.node),
            codemirror_mode: 'javascript'
        },
        child_process: [],
    });
}

module.exports = nodeKernel;



#### TODO: test node kernel on my own notebooks



#### TODO: demonstrate adding a feature like magics and suppressing messages



## TODO: 

add many kernels with small definition files

list the many features of different kernels and update the kernel list on jupyter github




TODO: user input and stdout streams

TODO: make a processing kernel https://github.com/processing/p5.js/wiki/p5.js,-node.js,-socket.io

Good examples: 

https://github.com/Calysto/calysto_processing/blob/master/calysto_processing/kernel.py



TODO: use magics parser from jupyter-nodejs, does something with splitting up lines, convert magics to babel and run along side the actual code?

TODO: transpile the kernel in to the native language using AST translations, use command line REPL or native ZMQ

https://jupyter-client.readthedocs.io/en/stable/kernels.html

TODO: use syntax highlighter to "train" a transpiler to write socket based kernels in any language, at least 

From Google:
- BROWSER
- C#
- GO
- JAVA
- NODE.JS
- PHP
- PYTHON
- RUBY


TODO: collect a list of kernels to support to make a codepen, coder pad clone.

https://pypi.org/search/?q=&o=&c=Framework+%3A%3A+IPython

From code pad:

- Bash
- C
- C#
- C++
- Clojure
- CoffeeScript
- Elixir
- Erlang
- F#
- Go
- HTML/CSS/JS
- Haskell
- Java
- JavaScript
- Kotlin
- Markdown
- MySQL
- OCaml
- Objective-C
- PHP
- Perl
- Plain Text
- PostgreSQL
- Python 2
- Python 3
- R
- Ruby
- Rust
- Scala
- Swift 5
- Visual Basic

From Jupyter:
- APL (Dyalog)
- Fortran 2008/2015
- Ansible 2.x
- Pyspark (Python 2 & 3), Spark (Scala), SparkR (R)
- python >= 3.3
- python 2.7, >= 3.3
- julia >= 0.3
- ghc >= 7.6
- ruby >= 2.1
- nodejs >= 0.10
- coffeescript >= 1.7
- livescript >= 1.5
- C# 4.0+
- R 3.2
- Any
- PARI/GP >= 2.9
- F#
- Go >= 1.8
- Go >= 1.6
- Go >= 1.9
- Go >= 1.4
- Scala
- Scala>=2.10
- Erlang
- Torch 7 (LuaJIT)
- Elixir >= 1.5
- Erlang >= 19, Elixir >= 1.4, LFE 1.2
- Aldor
- OCaml >= 4.01
- OCaml >= 4.02
- Forth
- Forth
- Perl 5
- Perl 6.c
- Perl 6
- Perl 6.C
- PHP >= 5.4
- PHP >= 7.0.0
- Octave
- Scilab
- Matlab
- bash
- zsh 5.3
- Mac Os X
- Windows
- Clojure >= 1.7
- Clojure
- Clojure 1.8
- Hy
- Hy
- redis
- io.js
- Babel
- multiple
- Mathics
- Wolfram Mathematica
- Lua
- Purescript
- Lua
- Lua
- Scheme
- Processing.js >= 2
- IDL
- Mochi
- Lua
- Scala, Python, R
- Skulpt Python
- bash
- python
- VPython
- Brainfuck
- Q
- Q
- Cryptol
- C++
- C++
- Xonsh
- Prolog
- SWI-Prolog
- Common Lisp
- Common Lisp
- Maxima
- YACAS
- Jython 2.7
- C++/python
- Gnuplot
- Tcl 8.5
- J 805-807 (J901beta)
- Jython>=2.7.0
- C
- TaQL
- Coconut
- Python 2.7 or >=3.4
- Python 2.7
- ARMv6 THUMB
- Python >=3.4
- NodeJS, Babel, Clojurescript
- Pike >= 7.8
- Typescript >= 2.0
- MATLAB >= 2016b
- Kotlin 1.1-M04 EAP
- Singular 4.1.0
- python >= 3.5, scala >= 2.11
- MIT Scheme 9.2
- Java + 9 scripting languages
- sbt >= 1.0.0
- ESP8266/ESP32
- Java 9
- Guile 2.0.12
- CircuitPython
- Stata
- Stata
- Racket >= 6.10
- dot/graphviz
- SQL
- HiveQL
- Rust >= 1.29.2
- StuPyd Programming Language
- Coq
- Cadabra2
- MongoDB
- Chapel
- Vim script
- graphviz/dot files
- Bash
- GAP >= 4.10


From visual studio code:

https://code.visualstudio.com/docs/languages/identifiers

- ABAP	abap
- Windows Bat	bat
- BibTeX	bibtex
- Clojure	clojure
- Coffeescript	coffeescript
- C	c
- C++	cpp
- C#	csharp
- CSS	css
- Diff	diff
- Dockerfile	dockerfile
- F#	fsharp
- Git	git-commit and git-rebase
- Go	go
- Groovy	groovy
- Handlebars	handlebars
- HTML	html
- Ini	ini
- Java	java
- JavaScript	javascript
- JavaScript React	javascriptreact
- JSON	json
- JSON with Comments	jsonc
- LaTeX	latex
- Less	less
- Lua	lua
- Makefile	makefile
- Markdown	markdown
- Objective-C	objective-c
- Objective-C++	objective-cpp
- Perl	perl and perl6
- PHP	php
- Powershell	powershell
- Pug	jade
- Python	python
- R	r
- Razor (cshtml)	razor
- Ruby	ruby
- Rust	rust
- SCSS	scss (syntax using curly brackets), sass (indented syntax)
- ShaderLab	shaderlab
- Shell Script (Bash)	shellscript
- SQL	sql
- Swift	swift
- TypeScript	typescript
- TypeScript React	typescriptreact
- TeX	tex
- Visual Basic	vb
- XML	xml
- XSL	xsl
- YAML	yaml



TODO: connect to M$ lang server

Long-term create a jupyter kernel for any REPL interface

looks like it is just passed to the command line in rust

https://github.com/google/evcxr/blob/602db3ef52bfddeb34608877bd72b9d0d112fa26/evcxr/src/module.rs

https://github.com/JuliaLang/IJulia.jl

matlab clone: https://mathics.angusgriffith.com/


Fix ijavascript not showing filename

https://nodejs.org/api/vm.html#vm_class_vm_script

Fix the problem with ijavascript not supporting %%

Go through this list and find demo code for every language and make sure it works with our REPL, minimize dependencies


Add HTML, CSS/SCSS kernels with some sort of visual output specially make for the file type

Create a kernels notebook



https://stackoverflow.com/questions/17637091/loading-ace-editor-from-cdn

Languages supported by Ace editor (AWS Cloud 9):

```html
<style>
#editor {
    position: relative;
    height:400px;
    width: 400px
}
</style>
<select id="mode" size="1">
    <option value="ace/mode/abap">abap</option>
    <option value="ace/mode/actionscript">actionscript</option>
    <option value="ace/mode/ada">ada</option>
    <option value="ace/mode/asciidoc">asciidoc</option>
    <option value="ace/mode/assembly_x86">assembly_x86</option>
    <option value="ace/mode/autohotkey">autohotkey</option>
    <option value="ace/mode/batchfile">batchfile</option>
    <option value="ace/mode/c9search">c9search</option>
    <option value="ace/mode/c_cpp">c_cpp</option>
    <option value="ace/mode/clojure">clojure</option>
    <option value="ace/mode/cobol">cobol</option>
    <option value="ace/mode/coffee">coffee</option>
    <option value="ace/mode/coldfusion">coldfusion</option>
    <option value="ace/mode/csharp">csharp</option>
    <option value="ace/mode/css">css</option>
    <option value="ace/mode/curly">curly</option>
    <option value="ace/mode/d">d</option>
    <option value="ace/mode/dart">dart</option>
    <option value="ace/mode/diff">diff</option>
    <option value="ace/mode/django">django</option>
    <option value="ace/mode/dot">dot</option>
    <option value="ace/mode/ejs">ejs</option>
    <option value="ace/mode/erlang">erlang</option>
    <option value="ace/mode/forth">forth</option>
    <option value="ace/mode/ftl">ftl</option>
    <option value="ace/mode/glsl">glsl</option>
    <option value="ace/mode/golang">golang</option>
    <option value="ace/mode/groovy">groovy</option>
    <option value="ace/mode/haml">haml</option>
    <option value="ace/mode/handlebars">handlebars</option>
    <option value="ace/mode/haskell">haskell</option>
    <option value="ace/mode/haxe">haxe</option>
    <option value="ace/mode/html">html</option>
    <option value="ace/mode/html_ruby">html_ruby</option>
    <option value="ace/mode/ini">ini</option>
    <option value="ace/mode/jade">jade</option>
    <option value="ace/mode/java">java</option>
    <option value="ace/mode/javascript" selected>javascript</option>
    <option value="ace/mode/json">json</option>
    <option value="ace/mode/jsoniq">jsoniq</option>
    <option value="ace/mode/jsp">jsp</option>
    <option value="ace/mode/jsx">jsx</option>
    <option value="ace/mode/julia">julia</option>
    <option value="ace/mode/latex">latex</option>
    <option value="ace/mode/less">less</option>
    <option value="ace/mode/liquid">liquid</option>
    <option value="ace/mode/lisp">lisp</option>
    <option value="ace/mode/livescript">livescript</option>
    <option value="ace/mode/logiql">logiql</option>
    <option value="ace/mode/lsl">lsl</option>
    <option value="ace/mode/lua">lua</option>
    <option value="ace/mode/luapage">luapage</option>
    <option value="ace/mode/lucene">lucene</option>
    <option value="ace/mode/makefile">makefile</option>
    <option value="ace/mode/markdown">markdown</option>
    <option value="ace/mode/matlab">matlab</option>
    <option value="ace/mode/mushcode">mushcode</option>
    <option value="ace/mode/mushcode_high_rules">mushcode_high_rules</option>
    <option value="ace/mode/mysql">mysql</option>
    <option value="ace/mode/objectivec">objectivec</option>
    <option value="ace/mode/ocaml">ocaml</option>
    <option value="ace/mode/pascal">pascal</option>
    <option value="ace/mode/perl">perl</option>
    <option value="ace/mode/pgsql">pgsql</option>
    <option value="ace/mode/php">php</option>
    <option value="ace/mode/powershell">powershell</option>
    <option value="ace/mode/prolog">prolog</option>
    <option value="ace/mode/properties">properties</option>
    <option value="ace/mode/python">python</option>
    <option value="ace/mode/r">r</option>
    <option value="ace/mode/rdoc">rdoc</option>
    <option value="ace/mode/rhtml">rhtml</option>
    <option value="ace/mode/ruby">ruby</option>
    <option value="ace/mode/rust">rust</option>
    <option value="ace/mode/sass">sass</option>
    <option value="ace/mode/scad">scad</option>
    <option value="ace/mode/scala">scala</option>
    <option value="ace/mode/scheme">scheme</option>
    <option value="ace/mode/scss">scss</option>
    <option value="ace/mode/sh">sh</option>
    <option value="ace/mode/snippets">snippets</option>
    <option value="ace/mode/sql">sql</option>
    <option value="ace/mode/stylus">stylus</option>
    <option value="ace/mode/svg">svg</option>
    <option value="ace/mode/tcl">tcl</option>
    <option value="ace/mode/tex">tex</option>
    <option value="ace/mode/text">text</option>
    <option value="ace/mode/textile">textile</option>
    <option value="ace/mode/toml">toml</option>
    <option value="ace/mode/twig">twig</option>
    <option value="ace/mode/typescript">typescript</option>
    <option value="ace/mode/vbscript">vbscript</option>
    <option value="ace/mode/velocity">velocity</option>
    <option value="ace/mode/verilog">verilog</option>
    <option value="ace/mode/xml">xml</option>
    <option value="ace/mode/xquery">xquery</option>
    <option value="ace/mode/yaml">yaml</option>
</select>
<div id="editor">function foo(items) { 
  var x = "All this is syntax highlighted"; 
  return x;
}</div>
<script src="//ajaxorg.github.io/ace-builds/src-min-noconflict/ace.js" type="text/javascript" charset="utf-8"></script>
<script>
    var editor = ace.edit("editor");
    editor.getSession().setUseWorker(false);
    editor.setTheme("ace/theme/monokai");
    editor.getSession().setMode("ace/mode/javascript");

    $('#mode').on('change', function (ev) {
        var mode = $('option:selected').attr('value');
        //console.log(mode)
        editor.getSession().setMode(mode);
    });
</script>
```


In [ ]:

// TODO: derrive from 
//  https://github.com/takluyver/bash_kernel/blob/master/bash_kernel/kernel.py
function bashKernel (config, options) {
    var kernel = replKernel({
        kernel_config: config,
        start_options: options,
        child_process: ['bash', '-c', `
echo "starting bash kernel";
do_execute() {
    request=$1
    return $(eval "$request")
}

do_complete() {
    request=$1
    return $(bash -n -c "$request") 
}

# equivilent of process.stdin.resume();
while [ true ]; do
    read -u $NODE_CHANNEL_FD request
    
    echo $request
done

`]
    });
    return kernel;
}

function bashLanguage() {
    return languageInterface(Object.assign({
        name: 'bash',
        // TODO: use the language kernel to execute instead
        version: require('child_process').execSync(`
bash --version | grep "bash" | cut -f 4 -d " " | cut -d "-" -f 1  | cut -d "(" -f 1
`).toString().trim(),
        file_extension: '.sh',
        mimetype: 'text/x-sh',
        codemirror_mode: 'shell'
    }, language_info));
}


## a couple of tools


### decode encode ipython zmq protocol?

translate between standard objects and the wire protocol message, using the first key of the object as the message type like `{execute_request: {content: etc}}`

In [ ]:
var util = require('util');
var zmq = require("jmp").zmq;
var crypto = require('crypto');
var uuid = require('uuid/v4');

// uuids, delim, hmac, header, parent_header, metadata, content
var DELIM = '<IDS|MSG>'
function parseMessage() {
    const strs = [].map.call(arguments, a => a.toString())
    let i
    for (i=0; i<strs.length; i++) {
      if (strs[i] === DELIM) {
        break
      }
    }
    const uuids = [].slice.call(arguments, 0, i)
    const args = strs.slice(i + 2).map(a => JSON.parse(a))
    let [header, parent, metadata, content] = args
    var result = {};
    result[header.msg_type] = {content, header, metadata, parent};
    return result;
}

function hash(string, key) {
    const hmac = crypto.createHmac('sha256', key)
    hmac.update(string)
    const res = hmac.digest('hex')
    return res
}

function json(data) {
    return JSON.stringify(data).replace('\ufdd0', '\\ufdd0')
}

function collapseMessage(key, message) {
    var msg_type = Object.keys(message)[0];
    var {header, parent, metadata, content} = message[msg_type];
    var header = Object.assign({}, parent || {}, {
        msg_id: uuid(),
        msg_type: msg_type
    })
    const toHash = [
      json(header || {}),
      json(parent || {}),
      json(metadata || {}),
      json(content || {})]
    const hmac = hash(toHash.join(''), key)
    return ((parent || {}).uuids || []).concat([DELIM, hmac]).concat(toHash);
}

module.exports = {
    parseMessage,
    collapseMessage,
}


### bind wire sockets?




In [ ]:

function setupSockets(config) {
    const sockets = {
        control: {
            port: config.control_port,
            type: 'xrep',
        },
        shell: {
            port: config.shell_port,
            type: 'xrep',
        },
        stdin: {
            port: config.stdin_port,
            type: 'router',
        },
        iopub: {
            port: config.iopub_port,
            type: 'pub',
        },
        heartbeat: {
            port: config.hb_port,
            type: 'rep',
        }
    }
    var keys = Object.keys(sockets);
    return Promise.all(keys.map(s => setupSocket.apply(null, [sockets[s], config])))
        .then(sockets => sockets.reduce((obj, socket, i) =>
                                        (obj[keys[i]] = socket, obj), {}))
}

function setupSocket(config, general) {
    const sock = zmq.socket(config.type);
    const addr = general.transport + '://' + general.ip + ':' + config.port
    return util.promisify(sock.bind.bind(sock))(addr)
        .then(() => sock)
}

module.exports = {
    setupSockets
}


### TODO: magics interpreter




#### suppress log messages categorically magic

Use lev matching and remove matches too far apart > .80 and too close together < .20 and group by what is left.

expected output 

```
%% suppress notebook
50 importing notebook messages suppressed
```

At the end of the execution.

